# Compute principal components

In this notebook, we compute principal components using the variants for the AoU and UKB participants.

# Setup 

In [ ]:
from datetime import datetime
import hail as hl
import os
import time

In [ ]:
from bokeh.io import show, output_notebook
from bokeh.layouts import gridplot
output_notebook()

## Define constants

<div class="alert alert-block alert-info">
<b>Note:</b> These matrix tables were created via notebook 'merge_variants'.
</div>

In [ ]:
MERGED_MT = [
    'gs://fc-secure-fd6786bf-6c28-4f33-ac30-3860fbeee5bb/data/merged/20210609/merged-filtered-chr1.mt',
    'gs://fc-secure-fd6786bf-6c28-4f33-ac30-3860fbeee5bb/data/merged/20210621/merged-filtered-chr2.mt',
    'gs://fc-secure-fd6786bf-6c28-4f33-ac30-3860fbeee5bb/data/merged/20210622/merged-filtered-chr3.mt',
    'gs://fc-secure-fd6786bf-6c28-4f33-ac30-3860fbeee5bb/data/merged/20210624/merged-filtered-chr4.mt',
    'gs://fc-secure-fd6786bf-6c28-4f33-ac30-3860fbeee5bb/data/merged/20210624/merged-filtered-chr5.mt',
    'gs://fc-secure-fd6786bf-6c28-4f33-ac30-3860fbeee5bb/data/merged/20210628/merged-filtered-chr6_chr7_chr8_chr9_chr10_chr11_chr12_chr13_chr14_chr15_chr16_chr17_chr18_chr19_chr20.mt',
    'gs://fc-secure-fd6786bf-6c28-4f33-ac30-3860fbeee5bb/data/merged/20210603/merged-filtered-chr21.mt',
    'gs://fc-secure-fd6786bf-6c28-4f33-ac30-3860fbeee5bb/data/merged/20210624/merged-filtered-chr22.mt',
    'gs://fc-secure-fd6786bf-6c28-4f33-ac30-3860fbeee5bb/data/merged/20210628/merged-filtered-chrX.mt',
    'gs://fc-secure-fd6786bf-6c28-4f33-ac30-3860fbeee5bb/data/merged/20210624/merged-filtered-chrY.mt'
]

In [ ]:
RESULT_BUCKET = os.getenv("WORKSPACE_BUCKET")
DATESTAMP = time.strftime('%Y%m%d')
TIMESTAMP = time.strftime('%Y%m%d_%H%M%S')
WORK_DIR = !pwd

# Outputs
SCORES_TAB = f'{os.getenv("WORKSPACE_BUCKET")}/data/merged/{DATESTAMP}/scores.tab'
SCORES_TSV = f'{os.getenv("WORKSPACE_BUCKET")}/data/merged/{DATESTAMP}/scores.tsv'
HAIL_LOG = f'{WORK_DIR[0]}/hail-compute-pcs-variants-{TIMESTAMP}.log'

## Check access

In [ ]:
for mt in MERGED_MT:
    !gsutil ls {mt}
    print('\n')

## Start Hail 

In [ ]:
# See also https://towardsdatascience.com/fetch-failed-exception-in-apache-spark-decrypting-the-most-common-causes-b8dff21075c
# See https://spark.apache.org/docs/2.4.7/configuration.html

EXTRA_SPARK_CONFIG = {
    # If set to "true", performs speculative execution of tasks. This means if one or more tasks are running slowly in a stage, they will be re-launched.
    'spark.speculation': 'true', # Default is false.
    
    # Fraction of tasks which must be complete before speculation is enabled for a particular stage.
    'spark.speculation.quantile': '0.95', # Default is 0.75

    # Default timeout for all network interactions. This config will be used in place of 
    # spark.core.connection.ack.wait.timeout, spark.storage.blockManagerSlaveTimeoutMs, 
    # spark.shuffle.io.connectionTimeout, spark.rpc.askTimeout or spark.rpc.lookupTimeout if they are not configured.
    'spark.network.timeout': '180s', # Default is 120s
        
    # (Netty only) Fetches that fail due to IO-related exceptions are automatically retried if this is set to a
    # non-zero value. This retry logic helps stabilize large shuffles in the face of long GC pauses or transient
    # network connectivity issues.
    'spark.shuffle.io.maxRetries': '10',  # Default is 3
    
    # (Netty only) How long to wait between retries of fetches. The maximum delay caused by retrying is 15 seconds
    # by default, calculated as maxRetries * retryWait.
    'spark.shuffle.io.retryWait': '15s',  # Default is 5s
    
    # Number of failures of any particular task before giving up on the job. The total number of failures spread
    # across different tasks will not cause the job to fail; a particular task has to fail this number of attempts.
    # Should be greater than or equal to 1. Number of allowed retries = this value - 1.
    'spark.task.maxFailures': '10', # Default is 4.

    # Number of consecutive stage attempts allowed before a stage is aborted.
    'spark.stage.maxConsecutiveAttempts': '10' # Default is 4.
}

In [ ]:
hl.init(spark_conf=EXTRA_SPARK_CONFIG,
        min_block_size=50,
        default_reference='GRCh38',
        log=HAIL_LOG)

Check the configuration.

In [ ]:
sc = hl.spark_context()
config = sc._conf.getAll()
config.sort()
config

# Read the matrix table

In [ ]:
merged = hl.read_matrix_table(MERGED_MT[0])

for i in range(1, len(MERGED_MT)):
    merged = merged.union_rows(hl.read_matrix_table(MERGED_MT[i]))

# Compute principal components

https://hail.is/docs/0.2/methods/genetics.html#hail.methods.hwe_normalized_pca

In [ ]:
start = datetime.now()
print(start)

In [ ]:
eigenvalues, scores, loadings = hl.hwe_normalized_pca(merged.GT)

In [ ]:
end = datetime.now()
print(end)
print(end - start)

In [ ]:
eigenvalues

In [ ]:
scores.write(SCORES_TAB)

In [ ]:
scores.describe()

In [ ]:
scores.show(10)

In [ ]:
scores.export(SCORES_TSV)

In [ ]:
type(loadings)

# Provenance

In [ ]:
print(datetime.now())

In [ ]:
!pip3 freeze